In [1]:
import pandas as pd
import numpy as np
import time

from selenium import webdriver 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
# Search page URL
url = 'https://abpharmacy.ca/'

# Save all our data in a global dataframe
df = pd.DataFrame()

In [3]:
# Load the browser
option = webdriver.ChromeOptions()
option.add_argument('-incognito')

browser = webdriver.Chrome(executable_path='../chromedriver', chrome_options=option)

C:\Users\Shannon\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [4]:
# Goes back and picks a new city, then clicks on the search button, then scrapes the table,
def goBackandPickCity(index):
    
    # Helper function to click on the bloody search button
    def clickTheSearchButton(element):
        """
        The search button is a pseudo element created by CSS and not attached to the DOM.
        Selenium can't find it because of that and can't click it. So we emulate a mouse
        click by offseting the x and y from the drop down menu right above the search btn.
        """
        actionChains = ActionChains(browser)
        actionChains.move_to_element_with_offset(element, 400, 60).click().perform()
    
    
    # Go to search page for AB Phramacy
    browser.get(url)
    
    # Grab the drop down menu
    selectPharm = Select(browser.find_element_by_xpath("//select[@name='uno']"))
    
    # Select pharmacy in 'I am looking for a:'
    selectPharm.select_by_index(1)
    
    # Click on the go button to bring the next drop down menu
    browser.find_element_by_xpath("//a[@class='btn btn-primary btn-sm btn-go']").click()
    
    # grab the selec city drop down menu
    selectCity = Select(browser.find_element_by_xpath("//select[@style='width: 515px;']"))
    
    # Select by index which city
    selectCity.select_by_index(index)
    
    # We regrab the city drop down menu as a web element so we can use its (x, y) location
    element = browser.find_element_by_xpath("//select[@style='width: 515px;']")
    
    # Click the search button
    clickTheSearchButton(element)
    
    # Give a second for the page to load
    time.sleep(1)
    
    # Grabs the info from the pages and saves it into the global df
    return scrapePages()

In [5]:
# Scrapes the table and the pages after it
def scrapePages():
    
    # Create a temporary df to store our data
    tempDf = pd.DataFrame()
    
    # Loop through each pages until we each the end
    while(True):
        
        # Grab the table element that has all the pharmacies
        table = browser.find_element_by_css_selector(".table")

        # Grab all the table rows in that table
        trList = table.find_elements_by_tag_name("tr")

        # Drop the header and loop through each row
        for tr in trList[1:]:

            # Grab the relevant fields we want
            name = tr.find_elements_by_tag_name("td")[0].text
            manager = tr.find_elements_by_tag_name("td")[1].text
            info = tr.find_elements_by_tag_name("td")[2].text

            # Package the data into a dict
            data = {'Name': name,
                    'Manager': manager,
                    'Info': info}

            # Append the dict to the main dataframe
            tempDf = tempDf.append(data, ignore_index=True)

        # Try and click the next button, if not then we reached the end
        try:
            nextButton = browser.find_element_by_class_name('next').find_element_by_tag_name('a').click()
        except:
            # No more next pages; at the end
            break

        # Wait one second to let the page load
        time.sleep(1)
        
    
    return tempDf

In [6]:
"""
The main loop that uses the two previous functions to loop through all the pharmacies 
"""

# Go to search page for AB Phramacy
browser.get(url)

# Grab the drop down menu
selectPharm = Select(browser.find_element_by_xpath("//select[@name='uno']"))

# Select pharmacy in 'I am looking for a:'
selectPharm.select_by_index(1)

# Click on the go button to bring the next drop down menu
browser.find_element_by_xpath("//a[@class='btn btn-primary btn-sm btn-go']").click()

# grab the selec city drop down menu
selectCity = Select(browser.find_element_by_xpath("//select[@style='width: 515px;']"))

for i in range(1, len(selectCity.options)):
    df = df.append(goBackandPickCity(i))
    

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.44.609538 (b655c5a60b0b544917107a59d4153d4bf78e1b90),platform=Windows NT 10.0.17763 x86_64)


In [ ]:
df.to_csv('./csv/AB Pharmacy.csv', index=False)